In [1]:
# Importing the libraries
import sqlite3
import random

In [2]:
%load_ext sql

# Connecting to the database

In [3]:
# Connecting to a sqlite database
%sql sqlite:///lab3.db 

In [4]:
#Checking the tables
%sql SELECT * FROM Student

 * sqlite:///lab3.db
Done.


SId,Sname,Sage,Sgender
01,Peter,1990-01-01,M
02,Sam,1990-12-21,M
03,Lee,1990-05-20,M
04,June,1990-08-06,M
05,Eda,1991-12-01,F
06,Eloise,1992-03-01,F
07,Silas,1989-07-01,F
09,Hazel,2001-12-20,F
10,Oscar,2002-12-25,F
11,Oscar,2002-12-30,F


# Q1: Find the student id that has all the grades in between 80 and 90.

In [6]:
%%sql
SELECT DISTINCT SId 
FROM Grade
WHERE SId NOT IN
    (SELECT SId
    FROM Grade
    WHERE score < 80 or score > 90
);

 * sqlite:///lab3.db
Done.


SId
03


In [7]:
# Checking the Grade table to see if the answer makes sense
%sql SELECT * FROM Grade

 * sqlite:///lab3.db
Done.


SId,CId,score
01,01,80
01,02,90
01,03,99
02,01,70
02,02,60
02,03,80
03,01,80
03,02,80
03,03,80
04,01,50


# Q02: Find the students, who earned a higher grade in Calculus than his/her History class.

In [8]:
# Check the code of each course
%sql SELECT * FROM Course

 * sqlite:///lab3.db
Done.


CId,Cname,PId
01,History,02
02,Calculus,01
03,CS-I,03


## Temporary tables that will be used in the JOIN

In [9]:
%%sql 
/* Creating temporary tables: one only for calculus scores and the other only for history scores */

CREATE TEMPORARY TABLE temp_calculus as 
SELECT DISTINCT s.Sname, a.SId, a.score as Calculus
FROM Student s, Grade a
WHERE s.SId = a.SId AND a.CId = '02';

CREATE TEMPORARY TABLE temp_history as 
SELECT DISTINCT s.Sname, g.SId, g.score as History 
FROM Student s, Grade g
WHERE g.CId = '01';


/* Joining both tables and selecting only the ones with Calculus score > than History */

SELECT DISTINCT c.Sname, c.Calculus, h.History
FROM temp_calculus c
INNER JOIN 
(SELECT * FROM temp_history) AS h
ON c.SId = h.SId
WHERE Calculus > History;

 * sqlite:///lab3.db
Done.
Done.
Done.


Sname,Calculus,History
Peter,90,80
Eda,87,76


In [10]:
%%sql
DROP TABLE temp_calculus;
DROP TABLE temp_history

 * sqlite:///lab3.db
Done.
Done.


[]

# Q03: Find the students, who earned a more than 70 average grade across all the courses he/she has taken.

In [70]:
%%sql

SELECT s.Sname, ROUND(AVG(score),1)
FROM Student s, Grade g
WHERE s.SId = g.SId
GROUP BY g.SId
HAVING AVG(score) > 70

 * sqlite:///lab3.db
Done.


Sname,"ROUND(AVG(score),1)"
Peter,89.7
Lee,80.0
Eda,81.5
Silas,93.5


# Q04: Find the students that did not take any courses taught by Professor Everett.

In [21]:
%%sql
/*Find the PId from each instructor*/
SELECT * FROM Instructor 

 * sqlite:///lab3.db
Done.


PId,Pname
01,Everett
02,Miles
03,June


In [14]:
%%sql
SELECT DISTINCT s.Sname
FROM Student s, Course c, Grade g
WHERE s.SId = g.SId AND c.CId = g.CId
AND g.SId NOT IN(
    SELECT g1.SId
    FROM Grade g1, Course c1
    WHERE g1.CId = c1.CId AND c1.PId = '01')

 * sqlite:///lab3.db
Done.


Sname
Eloise


# Q05: Find the students, who took the course Calculus and have earned a grade, at least, higher than all students who took History course. Return the results in a descending order of grade.

In [38]:
%%sql
SELECT s.Sname, g.score
FROM Student s, Grade g, Course c
WHERE s.SId = g.SId AND g.CId = c.CId 
AND c.Cname = 'Calculus' AND g.score > (
    SELECT MAX(g1.score) as maxHistScore
    FROM Grade g1, Course c1
    WHERE g1.CId = c1.CId AND c1.Cname = 'History'
    ORDER BY score DESC)
ORDER BY score DESC;

 * sqlite:///lab3.db
Done.


Sname,score
Peter,90
Silas,89
Eda,87


In [34]:
%%sql
/* This query is part of the query above: it select the MAX grade from the History course. If we take the MAX off, it shows all grades */
SELECT MAX(g.score) as maxHistScore
FROM Grade g, Course c
WHERE g.CId = c.CId AND c.Cname = 'History'
ORDER BY score DESC; 

 * sqlite:///lab3.db
Done.


maxHistScore
80


In [39]:
%%sql
/* In case you want to check all scores of a certain course, or any other filter*/
SELECT s.Sname, c.Cname, g.score
FROM Student s, Course c, Grade g
WHERE s.SId = g.SId AND c.CId = g.CId and Cname = 'Calculus'
ORDER BY score DESC

 * sqlite:///lab3.db
Done.


Sname,Cname,score
Peter,Calculus,90
Silas,Calculus,89
Eda,Calculus,87
Lee,Calculus,80
Sam,Calculus,60
June,Calculus,30


# Q06: Find the students, who did not take all the courses as presented in the Course table.

In [53]:
%%sql
SELECT s.Sname, COUNT(g.CId) as coursesTaken
FROM Student s, Grade g
WHERE s.SId = g.SId
GROUP BY (s.Sname)
HAVING COUNT(g.CId) < ( SELECT COUNT(c.Cname) as countCourses FROM Course c);


 * sqlite:///lab3.db
Done.


Sname,coursesTaken
Eda,2
Eloise,2
Silas,2


In [52]:
%%sql
/* This query is part of the query above: it COUNTS the number of courses of table COURSES */
SELECT COUNT(c.Cname) as countCourses
FROM Course c ;

 * sqlite:///lab3.db
Done.


countCourses
3


In [56]:
%%sql
/* This query is part of the query above: it shows the number of courses taken by each student */
SELECT s.Sname, COUNT(g.CId) as coursesTaken
FROM Student s, Grade g
WHERE s.SId = g.SId
GROUP BY (s.Sname)

 * sqlite:///lab3.db
Done.


Sname,coursesTaken
Eda,2
Eloise,2
June,3
Lee,3
Peter,3
Sam,3
Silas,2


# Q07: Calculate the average grades for Male and Female students.

In [16]:
%%sql
SELECT s.Sgender, ROUND(AVG(g.score),2) as avgGrade
FROM Student s, Grade g
WHERE s.SId = g.SId
GROUP BY s.Sgender

 * sqlite:///lab3.db
Done.


Sgender,avgGrade
F,69.17
M,68.25


# Q08: Find the courses with highest enrollment.

In [22]:
%%sql
SELECT g.CId, COUNT(g.SId) as studentEnrolled
FROM Grade g
GROUP BY g.CId
ORDER BY studentEnrolled DESC;

 * sqlite:///lab3.db
Done.


CId,studentEnrolled
03,6
02,6
01,6


In [18]:
%%sql
SELECT g.CId, COUNT(g.SId) as studentEnrolled
FROM Grade g
GROUP BY g.CId
ORDER BY studentEnrolled DESC;

 * sqlite:///lab3.db
Done.


CId,studentEnrolled
03,6
02,6
01,6


# Q09: For each course, find the students who earned more than its average grade.

## First, I will build a temporary table with the avg score per course. Then I use this table in the complete query

In [93]:
%%sql
CREATE TEMPORARY TABLE temp_avgScore as 
SELECT c.Cname, ROUND(AVG(g.score),1) as avgScore
FROM Course c, Grade g
WHERE c.CId = g.CId
GROUP BY g.CId;

 * sqlite:///lab3.db
Done.


[]

In [94]:
%%sql
SELECT DISTINCT s.Sname, c.Cname, ROUND(g.score,1) as score
FROM Student s, Grade g, Course c, temp_avgScore t
WHERE s.SId = g.SId AND g.CId = c.CId AND g.score > t.avgScore
ORDER BY c.Cname;

 * sqlite:///lab3.db
Done.


Sname,Cname,score
Peter,CS-I,99.0
Sam,CS-I,80.0
Lee,CS-I,80.0
Silas,CS-I,98.0
Peter,Calculus,90.0
Lee,Calculus,80.0
Eda,Calculus,87.0
Silas,Calculus,89.0
Peter,History,80.0
Sam,History,70.0


In [92]:
%%sql
DROP TABLE temp_avgScore;

 * sqlite:///lab3.db
Done.


[]

In [101]:
%%sql
/*Checking the values of the AVG per Course table*/
SELECT c.Cname, ROUND(AVG(g.score),1) as avgScore
FROM Course c, Grade g
WHERE c.CId = g.CId
GROUP BY g.CId

 * sqlite:///lab3.db
Done.


Cname,avgScore
History,64.5
Calculus,72.7
CS-I,68.5


# Q10: Please add a column "GPA" in the student table and calculate the GPA for each student. Based on the grade, >=90 is A (4.0), >=87 is A- (3.7), >=83 is B+ (3.3), >=80 is B (3.0), >=77 is B- (2.7), >=70 is C (2.0), <=70 and>60 is D (1.0) and <=60 is F (0).


# First, I will create a temp table with each student's average score. Then I build a new attribute column, and update using CASE.

In [130]:
%%sql 
/* ========= TEMPORARY TABLE  =============*/
CREATE TEMPORARY TABLE temp_gpa as 
SELECT s.Sname, ROUND(AVG(g.score),1) as avgScore
FROM Student s, Grade g
WHERE s.SId = g.SId
GROUP BY s.Sname
ORDER BY avgScore DESC;

/* ========= ADD a NEW COLUMN on a table =============*/
ALTER TABLE temp_gpa
ADD 'GPA' FLOAT;

/* ========= UPDATE this new column with gpa values ========= */
UPDATE temp_gpa
set GPA = (CASE
           WHEN avgScore > 90
           THEN 'A (4.0)'
           WHEN avgScore > 87
           THEN 'A- (3.7)'
           WHEN avgScore > 83
           THEN 'B+ (3.3)'
           WHEN avgSCore > 80
           THEN 'B (3.0)'
           WHEN avgSCore > 77
           THEN 'B- (2.7)'
           WHEN avgSCore > 70
           THEN 'C (2.0)'
           WHEN avgSCore > 60
           THEN 'D (1.0)'
           WHEN avgScore <= 60
           THEN 'F (0)'
           END);

/* ========= Finally, output the final query ========= */
SELECT * FROM temp_gpa;
           

 * sqlite:///lab3.db
Done.
Done.
Done.
Done.


Sname,avgScore,GPA
Silas,93.5,A (4.0)
Peter,89.7,A- (3.7)
Eda,81.5,B (3.0)
Lee,80.0,B- (2.7)
Sam,70.0,D (1.0)
June,33.3,F (0)
Eloise,32.5,F (0)


In [129]:
%sql DROP TABLE temp_gpa;

 * sqlite:///lab3.db
Done.


[]

In [ ]:
%%sql 
SELECT s.sname
FROM Sailors s, Reserves r
WHERE s.sid = r.sid AND r.bid = 110